In [1]:
import sys
sys.path.append('..')

import torch
from torch.nn import BCELoss
from torch.optim import Adam
from torchvision.models import resnet18

from lib.models import SimpleClassifier, ResNet, get_resnet
from lib.trainer import Trainer
from lib.utils import train_parse_args, get_data_loaders
from lib.constants import get_hyperparameters

In [2]:
params = get_hyperparameters()
params['batch_size'] = 8

if not params['disable_cuda'] and torch.cuda.is_available():
    params['device'] = torch.device('cuda:0')
else:
    params['device'] = torch.device('cpu')

print(params['device'])

cuda:0


In [ ]:
loaders = get_data_loaders(
    imgs_dir=params['imgs_dir'],
    labels_filename=params['labels_filename'],
    batch_size=params['batch_size'],
    n_imgs=params['n_imgs'])

model = ResNet()
model.cuda()

optimizer = Adam(model.parameters(), lr=3e-4)
criterion = BCELoss()
trainer = Trainer(params, model, optimizer, criterion)

trainer.run(loaders)

0 0.0
50 0.0
100 0.0
150 0.0
200 0.0
250 0.0
300 0.0
350 0.0
EPOCH 1
1 0 0.7949915938847991
1 50 -68.85168458776857
1 100 -69.84406900982778
1 150 -71.05803446998247
1 200 -70.31225252042091
1 250 -52.1906867144687
1 300 -70.33124684553205
1 350 -71.07867907375754
1 400 -70.45091675568125
1 450 -70.42707062428482
1 500 -70.23306798591602
1 550 -51.152580791672435
1 600 -69.88862513261608
1 650 -54.06697367673091
1 700 -48.62702180184563
1 750 -70.13350435112594
1 800 -68.9741015346648
1 850 -70.2816057204081
1 900 -70.65578842556069
1 950 -69.94202994434546
1 1000 -70.17325402727538
1 1050 -70.75365162981194
1 1100 -55.213988298262095
1 1150 -70.2338204345037
1 1200 -69.69232081106989
1 1250 -71.51508759376588
1 1300 -70.34760856944365
1 1350 -70.1735110012932
1 1400 -71.0694370378605
1 1450 -51.051310544362316
1 1500 -51.593034262906684
1 1550 -70.26335334323457
1 1600 -38.966004828862864
1 1650 -70.22460889411438
1 1700 -53.615691171915486
1 1750 -70.44348762863257
1 1800 -70.0002789